In [1]:
import random
import pandas as pd

random.seed(42)
N = 100
pos, neg = [], []

In [2]:
with open('chrX.fa') as s:
    chrx = ''.join(s.readlines()).replace('\n', '').upper()
chrx[:100]

'NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN'

In [3]:
with open('data/H3K4me3_A549.intersect_with_DeepZ.bed') as f:
    for line in f.readlines():
        ch, l, r = line.split()
        l, r = int(l), int(r)
        if ch == 'chrX' and r - 1 < len(chrx):
            L = l
            while L + N - 1 <= r and L + N <= len(chrx):
                pos.append(chrx[L - 1:L - 1 + N])
                L += N
print(N, len(pos))

100 224


In [4]:
import random
for i in range(len(pos)):
    neg.append([])
    for j in range(N):
        neg[-1].append(random.randrange(4))

In [5]:
from sklearn.model_selection import train_test_split

for i in range(len(pos)):
    tmp = []
    for j in range(len(pos[i])):
        tmp.append('ACGT'.index(pos[i][j]))
    pos[i] = tmp

pos_train, pos_test = train_test_split(pos, test_size=0.2)
neg_train, neg_test = train_test_split(neg, test_size=0.2)
len(pos_train)

179

In [6]:
X_train = pos_train + neg_train
X_test = pos_test + neg_test
y_train = [1] * len(pos_train) + [0] * len(neg_train)
y_test = [1] * len(pos_test) + [0] * len(neg_test)
print(y_test)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [7]:
print(len(X_train), len(X_train[0]))

358 100


In [8]:
import torch
import IPython
import sklearn
from tqdm.notebook import tqdm
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
criterion = torch.nn.MSELoss()

In [9]:
X_train = pos_train + neg_train
X_test = pos_test + neg_test
y_train = [1] * len(pos_train) + [0] * len(neg_train)
y_test = [1] * len(pos_test) + [0] * len(neg_test)

In [10]:
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Sequential(
           nn.Linear(100, 100),
           nn.LeakyReLU(),
           nn.Linear(100, 100),
           nn.LeakyReLU(),
           nn.Linear(100, 100),
           nn.LeakyReLU(),
           nn.Linear(100, 1)
        )

    def forward(self, x):
        return self.linear(x)
    
model = Model()

In [11]:
optimizer = torch.optim.Adam(model.parameters(), lr=2e-3, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.85)
        
from torch.utils import data as data
train_loader = data.DataLoader(data.TensorDataset(torch.Tensor(np.array(X_train)), torch.Tensor(np.array(y_train))), batch_size = 20)
test_loader = data.DataLoader(data.TensorDataset(torch.Tensor(np.array(X_test)), torch.Tensor(np.array(y_test))), batch_size = 20)
model.train()
for epoch in range(42):
    for tracks, labels in tqdm(train_loader):
        optimizer.zero_grad()
        outputs = model(tracks)
        outputs = outputs.squeeze(1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    with torch.no_grad():
        model.eval()
        tr_pr = model(torch.Tensor(X_train)).detach().numpy()
        test_pr = model(torch.Tensor(X_test)).detach().numpy()
        tr_score = sklearn.metrics.mean_squared_error(y_train, tr_pr)
        test_score = sklearn.metrics.mean_squared_error(y_test, test_pr)
        print('Train:', tr_score, 'Test:', test_score)


  0%|          | 0/18 [00:00<?, ?it/s]

Train: 0.5423728335580613 Test: 0.5468923121312705


  0%|          | 0/18 [00:00<?, ?it/s]

Train: 0.33948881521801566 Test: 0.3400793691528182


  0%|          | 0/18 [00:00<?, ?it/s]

...

Train: 0.24529415732387613 Test: 0.2560481627745132


  0%|          | 0/18 [00:00<?, ?it/s]

Train: 0.24207440596862329 Test: 0.25158863360666955


  0%|          | 0/18 [00:00<?, ?it/s]

Train: 0.24070797446792241 Test: 0.25164986226315644
